In [2]:
from os import path
import pandas as pd
import numpy as np
import gc
import requests
import shutil
from pprint import pprint
import pickle

In [17]:
# Obtain the NAM data (split into training and testing)
# NOTE: must have run the "nam_data_preparation.ipynb" notebook beforehand

# Read the training and testing data from disk
cleaned_data = "cleaned_data/"
training_data_filename = "nam_training_"
testing_data_filename = "nam_testing_"
extension = ".pkl.gz"
nam_training_df = None
for i in range(32):
    training_part = pd.read_pickle(cleaned_data + training_data_filename + str(i) + extension, compression='gzip')
    nam_training_df = training_part if nam_training_df is None else nam_training_df.append(training_part)
    del training_part
    gc.collect()

In [18]:
# Print the training dataset out
pprint(nam_training_df[:5])
pprint(nam_training_df[-5:])

                                                                data_objects
timestamps                                                                  
2015-01-01 01:00:00-06:00  [[[0.0, 290.43628, 0.0, 73.0, 101516.0, 295.77...
2015-01-01 02:00:00-06:00  [[[0.0, 290.8083, 0.0, 74.0, 101502.0, 295.740...
2015-01-01 03:00:00-06:00  [[[0.0, 290.99026, 0.0, 75.0, 101467.0, 295.66...
2015-01-01 04:00:00-06:00  [[[0.0, 290.83884, 0.0, 75.0, 101436.0, 295.67...
2015-01-01 05:00:00-06:00  [[[0.0, 290.83582, 0.0, 75.0, 101412.0, 295.64...
                                                                data_objects
timestamps                                                                  
2018-12-31 17:00:00-06:00  [[[0.0, 283.83853, 268.59998, 58.5307, 101409....
2018-12-31 18:00:00-06:00  [[[0.0, 284.02292, 77.4, 58.640553, 101367.984...
2018-12-31 19:00:00-06:00  [[[0.0, 284.31033, 51.6, 59.009323, 101398.63,...
2018-12-31 20:00:00-06:00  [[[0.0, 284.59775, 25.8, 59.378094, 101429.28,...

In [41]:
# Obtain the ERCOT data (reduced hourly and split into train/test in a 24:1 ratio)
training_data_filename = "ercot_wind_power.pkl"
training_expected_output_filename = "ercot_for_nam_training"
testing_expected_output_filename = "ercot_for_nam_testing"

# If we haven't already done this split, do it now and save the contents to disk
if not path.exists(cleaned_data + training_expected_output_filename + extension) or not path.exists(cleaned_data + testing_expected_output_filename + extension):
    print("cleaning ERCOT data for NAM purposes")

#     # Download the ERCOT dataset if needed
#     if not path.exists(training_data_filename):
#         TODO figure out how to correctly download this pickle file, because this code doesn't work
#         r = requests.get("http://storage.googleapis.com/gridmatic/roscoe/" + training_data_filename, stream=True)
#         pickle(r.raw, training_data_filename)
#         with open(training_data_filename, 'wb') as fin:
#             shutil.copyfileobj(r.raw, fin)
#             fin.close()

    # Get generically cleaned ERCOT data, not specifically set up for use with NAM
    ercot_wind_power = pd.read_pickle("ercot_wind_power.pkl")
 
    # ERCOT data is every 15 minutes, while the NAM data is hourly
    # We therefore only use the ERCOT data on the hour, when working with NAM
    hourly_ercot = ercot_wind_power[::4]

    # Small change: NAM ends at 21:00 on Dec 31 2018, while ERCOT ends at 23:00
    # So cut out the last two entries of hourly_ercot
    hourly_ercot = hourly_ercot.iloc[:-2]

    # Split the ERCOT data to line up with the NAM data
    hourly_ercot['numerical_index'] = range(0, len(hourly_ercot))
    ercot_training_df = hourly_ercot[hourly_ercot['numerical_index'] % 25 != 0]
    ercot_testing_df = hourly_ercot[hourly_ercot['numerical_index'] % 25 == 0]
    del hourly_ercot
    gc.collect()
    ercot_training_df = ercot_training_df.drop('numerical_index', 1)
    ercot_testing_df = ercot_testing_df.drop('numerical_index', 1)
    
    # Save the ERCOT data to file
    ercot_training_df.to_pickle(cleaned_data + training_expected_output_filename + extension, compression='gzip')
    ercot_testing_df.to_pickle(cleaned_data + testing_expected_output_filename + extension, compression='gzip')
    del ercot_training_df
    del ercot_testing_df
    gc.collect()

# Get the NAM-specific ERCOT data from disk
ercot_training_df = pd.read_pickle(cleaned_data + training_expected_output_filename + extension, compression='gzip')

In [42]:
# Print the training dataset out
pprint(ercot_training_df.iloc[:5,:3])
pprint(ercot_training_df.iloc[-5:,:3])

resource_code              ANACACHO_ANA  ASTRA_UNIT1  BCATWIND_WIND_1
timestamp                                                            
2015-01-01 01:00:00-06:00           0.0          NaN              0.0
2015-01-01 02:00:00-06:00           0.0          NaN              0.0
2015-01-01 03:00:00-06:00           0.0          NaN              0.0
2015-01-01 04:00:00-06:00           0.0          NaN              0.0
2015-01-01 05:00:00-06:00           0.0          NaN              0.0
resource_code              ANACACHO_ANA  ASTRA_UNIT1  BCATWIND_WIND_1
timestamp                                                            
2018-12-31 17:00:00-06:00      0.000000     33.30522         11.35381
2018-12-31 18:00:00-06:00      0.000000     36.93887         12.76649
2018-12-31 19:00:00-06:00      0.000000     38.98851         23.02475
2018-12-31 20:00:00-06:00      0.582387     39.37746         34.87904
2018-12-31 21:00:00-06:00      7.234724     39.16596         33.18754


In [3]:
# Get the NAM locations
# Locations corresponding to each of the NAM forecasts (140x140x2)
# Latitude and longitude were originally in the opposite order from the coordinate data for the wind farms
# That's why we use this secondary file
nam_locations = np.load("nam_locations_flipped.npy")

In [19]:
pprint(nam_locations)

array([[[-107.76918787,   23.53865642],
        [-107.65016388,   23.54891584],
        [-107.53112034,   23.5590795 ],
        ...,
        [ -91.35751774,   24.04493008],
        [ -91.23753108,   24.04193683],
        [ -91.11755031,   24.03884662]],

       [[-107.78044054,   23.64776964],
        [-107.6613129 ,   23.65803855],
        [-107.54216564,   23.66821161],
        ...,
        [ -91.35429032,   24.15450932],
        [ -91.23419745,   24.15151333],
        [ -91.11411049,   24.14842029]],

       [[-107.79171301,   23.75688688],
        [-107.67248153,   23.76716526],
        [-107.55323038,   23.7773477 ],
        ...,
        [ -91.35105718,   24.26409157],
        [ -91.23085791,   24.26109285],
        [ -91.11066457,   24.25799698]],

       ...,

       [[-109.51991989,   38.35614736],
        [-109.38480872,   38.36748391],
        [-109.24966885,   38.3787143 ],
        ...,
        [ -90.8547987 ,   38.91516188],
        [ -90.71827237,   38.9118593 ],
        [

In [15]:
# Downsampling
# Motivation: the NAM data exists as a 140*140*9 matrix for every single timestamp.
# This covers the entire 140 by 140 grid, with 9 variables per grid square
# While very useful data in theory, it requires a significant amount of processing power
# Here, we will "downsample" this data by extracting its most useful components

# One method of doing this is by taking the grid squares closest to the wind farm we want to train on.
# Wind farm locations are found using the data_name_to_coords file

# Get the ERCOT locations
data_name_to_coords = None
with open('data_name_to_coords.pkl', 'rb') as handle:
    data_name_to_coords = pickle.load(handle)
pprint(data_name_to_coords)

{'ANACACHO_ANA': [-100.15802, 29.215214],
 'ASTRA_UNIT1': [-102.071477, 34.786654],
 'BCATWIND_WIND_1': [-98.570491, 33.509188],
 'BLSUMMIT_BLSMT1_5': [-99.367467, 34.293473],
 'BLSUMMIT_BLSMT1_6': [-99.367467, 34.293473],
 'BORDAS2_JAVEL2_A': [-98.931877, 27.358092],
 'BORDAS2_JAVEL2_B': [-98.931877, 27.358092],
 'BORDAS2_JAVEL2_C': [-98.931877, 27.358092],
 'BORDAS_JAVEL18': [-98.931877, 27.358092],
 'BORDAS_JAVEL20': [-98.931877, 27.358092],
 'BRAZ_WND_WND1': [-101.146911, 32.949884],
 'BRAZ_WND_WND2': [-101.146911, 32.949884],
 'BRISCOE_WIND': [-101.374278, 34.371453],
 'BRTSW_BCW1': [-98.363595, 33.073455],
 'BUCKTHRN_UNIT1': [-98.353682, 32.376956],
 'BUCKTHRN_UNIT2': [-98.353682, 32.376956],
 'BUFF_GAP_UNIT1': [-100.115386, 32.298543],
 'BUFF_GAP_UNIT2_1': [-100.115386, 32.298543],
 'BUFF_GAP_UNIT2_2': [-100.115386, 32.298543],
 'BUFF_GAP_UNIT3': [-100.115386, 32.298543],
 'BULLCRK_WND1': [-101.605095, 32.906769],
 'BULLCRK_WND2': [-101.605095, 32.906769],
 'CALLAHAN_WND1': [-99

In [29]:
# Create a map from grid square coordinates to grid square number
# coord_to_square_number = { tuple(location):i for i,location in enumerate(np.reshape(nam_locations, (19600,2)))}
# pprint(coord_to_square_number)

numbers = range(19600)

# Prepare a nearest neighbor interpolator
from scipy.interpolate import NearestNDInterpolator
myInterpolator = NearestNDInterpolator(np.reshape(nam_locations, (19600,2)), numbers)

# Given a specific wind farm, return NAM data within the radius specified
def only_proximal_nam_data(original_data_frame, wind_farm_name, radius):
    # Find the center of the new subset of NAM data that we want
    farm_location = data_name_to_coords[wind_farm_name]
    print(farm_location)
    central_grid_square = myInterpolator(farm_location[0],farm_location[1])
    print(central_grid_square)
    central_grid_square = (central_grid_square % 140, central_grid_square // 140)
    print(central_grid_square)
    lower_bound_x = max(0,central_grid_square[0] - radius)
    upper_bound_x = min(140,central_grid_square[0] + radius) # Used in range which is non inclusive
    lower_bound_y = max(0,central_grid_square[1] - radius)
    upper_bound_y = min(139,central_grid_square[1] + radius) # Used in range which is non inclusive
    print(lower_bound_x)
    print(upper_bound_x)
    print(lower_bound_y)
    print(upper_bound_y)

#     central_grid_square = find_central_grid_square(data_name_to_coords[wind_farm_name])
    
    # Now a question is, how are we obtaining a subset of the NAM data?
    # Is this going to be vectorized or a loop?
    # Is this going to be with all NAM data simultaneously? Or done piecewise to avoid OOM?
#     proximal_nam_data = np.array()
    localized_training_df = original_data_frame['data_objects'].map(lambda full_data: full_data[range(lower_bound_x, upper_bound_x),range(lower_bound_y, upper_bound_y)])
    return localized_training_df
    

In [39]:
# Testing out the function to only return proximal NAM data, seems to be working
# puny_nam_training = nam_training_df.iloc[:10,:]
# # pprint(puny_nam_training)

# test_run_nam_training = only_proximal_nam_data(puny_nam_training,  'CEDROHIL_CHW2', 2)
# pprint(test_run_nam_training)
# pprint(test_run_nam_training.iloc[0].shape)

[-98.843784, 27.608743]
4696
(76, 33)
74
78
31
35


In [36]:
# Flatten each 140*140*9 array, and stack those to create a matrix of data points
# Then train on that + the vector of ERCOT data values for a single farm

from sklearn import linear_model

# Can't call method in apply, so instead have a wrapper function to call the method
def flatten_wrapper(np_array):
    return np_array.flatten()

def reshape_regression_data(df):
    regression_data = df.apply(flatten_wrapper).values
    regression_tuple = tuple(regression_data)
    del regression_data
    gc.collect()
    nam_matrix = np.stack(regression_tuple, axis = 0)
    del regression_tuple
    gc.collect()
    return nam_matrix

def reshape_other_data(other_data):
    other_data = other_data.values.reshape(other_data.shape[0], 1)
    return other_data

def run_linear_regression(data1, data2):
    lm = linear_model.LinearRegression()
    model = lm.fit(data1, data2)   
    return model

In [37]:
# Localize and Reshape the training dataframe
reshaped_nam_training = reshape_regression_data(only_proximal_nam_data(nam_training_df, 'ANACACHO_ANA', 3))
# Delete the original Pandas dataframe from memory
del nam_training_df
gc.collect()

[-100.15802, 29.215214]
6785
(65, 48)
62
68
45
51


0

In [43]:
# Reshape the other training dataframe
reshaped_ercot_training_0 =  reshape_other_data(ercot_training_df.iloc[:,0])
# Delete the original Pandas dataframe from memory
del ercot_training_df
gc.collect()

138

In [44]:
nam_training_model = run_linear_regression(reshaped_nam_training, reshaped_ercot_training_0)
del reshaped_nam_training
del reshaped_ercot_training_0
gc.collect()

20

In [45]:
print(nam_training_model.coef_)

[[-1.01937786e-01 -8.72202337e-01  1.91381783e-03  1.49262398e-01
   4.47013676e-02  1.04719889e+00 -3.09559051e-03 -7.04252064e-01
  -2.31820047e-01  7.60008931e-01  6.63485527e-01 -1.87379122e-03
  -7.57489428e-02 -4.33148667e-02 -1.14215565e+00  1.90939754e-03
   2.20099792e-01  2.08827555e-01 -1.36821294e+00  2.62580931e-01
   7.72520900e-04 -7.05151930e-02 -2.21049003e-02 -1.17932212e+00
  -1.95845612e-03  4.03062791e-01 -7.69560784e-02  4.83372033e-01
  -1.48556128e-01 -2.66874582e-03 -1.68638900e-02  5.93578964e-02
   1.65483546e+00 -5.93669713e-04  8.27404737e-01  1.82676244e+00
   6.82609856e-01 -1.35125652e-01 -2.81143188e-03  5.71908951e-02
  -4.88084555e-02  3.27686548e-01  1.36613846e-03 -1.40101707e+00
  -2.91446090e+00  1.82623714e-01  1.78244829e-01  3.87400389e-04
  -6.69064522e-02  6.88600540e-03 -9.33087349e-01 -8.08507204e-04
   3.20182234e-01  1.51046586e+00]]


In [46]:
# Get the NAM testing information
nam_testing_df = None
for i in range(32):
    testing_part = pd.read_pickle(cleaned_data + testing_data_filename + str(i) + extension, compression='gzip')
    nam_testing_df = testing_part if nam_testing_df is None else nam_testing_df.append(testing_part)
del testing_part
gc.collect()

0

In [47]:
pprint(nam_testing_df[:5])
pprint(nam_testing_df[-5:])

                                                                data_objects
timestamps                                                                  
2015-01-01 00:00:00-06:00  [[[0.0, 290.7572, 0.0, 75.0, 101402.0, 295.582...
2015-01-02 01:00:00-06:00  [[[0.0, 290.68527, 0.0, 77.0, 101398.0, 294.99...
2015-01-03 02:00:00-06:00  [[[0.0, 285.07336, 0.0, 59.0, 101692.0, 293.30...
2015-01-04 03:00:00-06:00  [[[0.0, 287.89383, 0.0, 68.0, 101669.0, 294.14...
2015-01-05 04:00:00-06:00  [[[0.0, 288.7345, 0.0, 66.0, 101515.0, 295.621...
                                                                data_objects
timestamps                                                                  
2018-12-27 06:00:00-06:00  [[[0.0, 286.81763, 0.0, 68.75848, 101341.82, 2...
2018-12-28 07:00:00-06:00  [[[0.0, 285.78033, 0.0, 59.046467, 101340.87, ...
2018-12-29 08:00:00-06:00  [[[0.0, 289.72476, 5.5, 79.63069, 101430.37, 2...
2018-12-30 09:00:00-06:00  [[[0.0, 283.95175, 161.0, 60.42068, 101588.39,...

In [48]:
# Get the ERCOT testing information
ercot_testing_df = pd.read_pickle(cleaned_data + testing_expected_output_filename + extension, compression='gzip')

In [49]:
pprint(ercot_testing_df.iloc[:5,:3])
pprint(ercot_testing_df.iloc[-5:,:3])

resource_code              ANACACHO_ANA  ASTRA_UNIT1  BCATWIND_WIND_1
timestamp                                                            
2015-01-01 00:00:00-06:00      0.000000          NaN         0.000000
2015-01-02 01:00:00-06:00      0.825478          NaN         0.000000
2015-01-03 02:00:00-06:00      5.294131          NaN         0.967167
2015-01-04 03:00:00-06:00     23.222590          NaN        35.644380
2015-01-05 04:00:00-06:00      6.198874          NaN         2.759169
resource_code              ANACACHO_ANA  ASTRA_UNIT1  BCATWIND_WIND_1
timestamp                                                            
2018-12-27 06:00:00-06:00     18.213490     40.30907        25.250760
2018-12-28 07:00:00-06:00      6.029018     30.74844         4.998176
2018-12-29 08:00:00-06:00      5.078777      0.00000         3.735878
2018-12-30 09:00:00-06:00      1.002264     31.68289         0.000000
2018-12-31 10:00:00-06:00      0.000000     39.24713        29.878060


In [50]:
# Reshape the training dataframe
reshaped_nam_testing = reshape_regression_data(only_proximal_nam_data(nam_testing_df, 'ANACACHO_ANA', 3))
# Delete the original Pandas dataframe from memory
del nam_testing_df
gc.collect()

[-100.15802, 29.215214]
6785
(65, 48)
62
68
45
51


7

In [51]:
# Reshape the training dataframe
reshaped_ercot_testing_0 = reshape_other_data(ercot_testing_df.iloc[:,0])
# Delete the original Pandas dataframe from memory
del ercot_testing_df
gc.collect()

20

In [52]:
print(nam_training_model.score(reshaped_nam_testing, reshaped_ercot_testing_0))
del reshaped_nam_testing
del reshaped_ercot_testing_0
gc.collect()

0.25510180462531085


20

In [53]:
import pickle
# Save the linear regression model to a file
filename = 'nam_basic_regression_model.sav'
pickle.dump(nam_training_model, open(filename, 'wb'))
# to later load the model from disk, run:
# loaded_model = pickle.load(open(filename, 'rb'))